In [2]:
from IPython.display import Image, display_png

import numpy as np
import pandas as pd

## 판다스 팬시 인덱싱
## 컬롬 필터
## 데이터 합치키

import xgboost as xgb
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, confusion_matrix
## 위 임포트한 평가지표
## 분류 모델은 모호하다
## confusion_matrix이걸 처리하는게 관건이다. 방점찾기 중요.
## [y,y , y,n] , [n,y , n,n]
## 정확도를 맹신하지마라
## 판단기준의 하한점으로 잡고 앙상불 모델을 잡고 , 아니면 svm을 잡고 설정.
## xgboost는 마이크로에서 분류모델을 하기 위해 만든 라이브러리. 사이킷런 보다 알고리즘이 쎈데 전처리를 더 많이 해야함. 책이 따로 있을 정도로 슈파스타.
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier

import warnings

warnings.filterwarnings("ignore")

##선형 분류 모델은

In [3]:
train = pd.read_csv("data/titanic_train.csv")
test = pd.read_csv("data/titanic_test.csv")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
print(train.isnull().sum())
print(test.isnull().sum())
#이문제는 na를 못지우게 만들어놓는다
#죽을까 살까 이진분류
#전처리를 다해야한다 짜증~

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [6]:
train["Age"] = train["Age"].fillna(train["Age"].median())
test["Age"] = test["Age"].fillna(test["Age"].median())
test["Fare"] = test["Fare"].fillna(test["Fare"].median())
train["Embarked"] = train["Embarked"].fillna("S")
##빈곳을 다 때려넣는다 
##타이타닉은 잘 안낸다. 외워서 하기때문에. 

In [7]:
print(train.isnull().sum())
print(test.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64


In [8]:
train["Sex"][train["Sex"] == "male"] = 0
train["Sex"][train["Sex"] == "female"] = 1
test["Sex"][test["Sex"] == "male"] = 0
test["Sex"][test["Sex"] == "female"] = 1

In [9]:
train["Embarked"][train["Embarked"] == "S"] = 0
train["Embarked"][train["Embarked"] == "C"] = 1
train["Embarked"][train["Embarked"] == "Q"] = 2
test["Embarked"][test["Embarked"] == "S"] = 0
test["Embarked"][test["Embarked"] == "C"] = 1
test["Embarked"][test["Embarked"] == "Q"] = 2

In [10]:
y = train.loc[:, ["Survived"]]
x = train.loc[:, ["Sex", "Age", "Pclass", "SibSp", "Fare", "Embarked"]]
print(x, y)

    Sex   Age  Pclass  SibSp     Fare Embarked
0     0  22.0       3      1   7.2500        0
1     1  38.0       1      1  71.2833        1
2     1  26.0       3      0   7.9250        0
3     1  35.0       1      1  53.1000        0
4     0  35.0       3      0   8.0500        0
..   ..   ...     ...    ...      ...      ...
886   0  27.0       2      0  13.0000        0
887   1  19.0       1      0  30.0000        0
888   1  28.0       3      1  23.4500        0
889   0  26.0       1      0  30.0000        1
890   0  32.0       3      0   7.7500        2

[891 rows x 6 columns]      Survived
0           0
1           1
2           1
3           1
4           0
..        ...
886         0
887         1
888         0
889         1
890         0

[891 rows x 1 columns]


In [11]:
y = train.loc[:, ["Survived"]]
x = train.loc[:, ["Sex", "Age", "Pclass", "SibSp", "Fare", "Embarked"]]
print(x, y)

    Sex   Age  Pclass  SibSp     Fare Embarked
0     0  22.0       3      1   7.2500        0
1     1  38.0       1      1  71.2833        1
2     1  26.0       3      0   7.9250        0
3     1  35.0       1      1  53.1000        0
4     0  35.0       3      0   8.0500        0
..   ..   ...     ...    ...      ...      ...
886   0  27.0       2      0  13.0000        0
887   1  19.0       1      0  30.0000        0
888   1  28.0       3      1  23.4500        0
889   0  26.0       1      0  30.0000        1
890   0  32.0       3      0   7.7500        2

[891 rows x 6 columns]      Survived
0           0
1           1
2           1
3           1
4           0
..        ...
886         0
887         1
888         0
889         1
890         0

[891 rows x 1 columns]


In [12]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, shuffle=True, random_state=0
)
print(x_train, y_train)
print(x_test, y_test)

    Sex   Age  Pclass  SibSp     Fare Embarked
857   0  51.0       1      0  26.5500        0
52    1  49.0       1      1  76.7292        1
386   0   1.0       3      5  46.9000        0
124   0  54.0       1      0  77.2875        0
578   1  28.0       3      1  14.4583        1
..   ..   ...     ...    ...      ...      ...
835   1  39.0       1      1  83.1583        1
192   1  19.0       3      1   7.8542        0
629   0  28.0       3      0   7.7333        2
559   1  36.0       3      1  17.4000        0
684   0  60.0       2      1  39.0000        0

[623 rows x 6 columns]      Survived
857         1
52          1
386         0
124         0
578         0
..        ...
835         1
192         1
629         0
559         1
684         0

[623 rows x 1 columns]
    Sex   Age  Pclass  SibSp      Fare Embarked
495   0  28.0       3      0   14.4583        1
648   0  28.0       3      0    7.5500        0
278   0   7.0       3      4   29.1250        2
31    1  28.0       1      1

In [13]:
x_train = x_train.astype("float32")
y_train = y_train.astype("float32")
x_test = x_test.astype("float32")
y_test = y_test.astype("float32")

In [14]:
rf = RandomForestClassifier

NameError: name 'bst' is not defined